In [ ]:
# model_utils.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import librosa
import os

# Константы
SAMPLE_RATE = 22050
N_MFCC = 40
N_MELS = 128
HOP_LENGTH = 512
DURATION = 30
SUPPORTED_FORMATS = {'.mp3', '.wav', '.ogg'}
MAX_FILE_SIZE = 20 * 1024 * 1024  # 20 MB

# Определение устройства
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Определение модели
class Enhanced_CNN_RNN(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        
        # Улучшенная CNN часть
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Dropout2d(0.25),
            
            nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Dropout2d(0.25),
            
            nn.Conv2d(64, 128, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Dropout2d(0.25),
            
            nn.Conv2d(128, 256, kernel_size=(3, 3), padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Dropout2d(0.25)
        )
        
        # Автоматический расчет размера после сверточных слоев
        with torch.no_grad():
            # ОБНОВЛЕНО: теперь используем правильную высоту 193 вместо 168
            dummy_input = torch.randn(1, 1, 193, 1300)  # (каналы, высота, ширина)
            dummy_output = self.conv(dummy_input)
            self.conv_output_channels = dummy_output.size(1)
            self.conv_output_height = dummy_output.size(2)
            self.conv_output_width = dummy_output.size(3)
            
            # Размер для GRU: каналы * высота
            self.gru_input_size = self.conv_output_channels * self.conv_output_height
            
            print(f"Автоматический расчет размеров:")
            print(f"  Каналы после сверток: {self.conv_output_channels}")
            print(f"  Высота после сверток: {self.conv_output_height}")
            print(f"  Ширина после сверток: {self.conv_output_width}")
            print(f"  Входной размер для GRU: {self.gru_input_size}")
        
        # RNN часть с автоматическим расчетом размера входа
        self.gru = nn.GRU(
            input_size=self.gru_input_size,  # Используем автоматический расчет
            hidden_size=256,
            batch_first=True,
            bidirectional=True,
            num_layers=2,
            dropout=0.3
        )
        
        # Полносвязные слои
        self.fc = nn.Sequential(
            nn.Linear(256 * 2, 128),  # Умножаем на 2 из-за bidirectional
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, n_classes)
        )
        
    def forward(self, x):
        # CNN
        x = self.conv(x)
        
        # Подготовка для RNN
        batch_size, channels, height, width = x.size()
        
        # Проверка совместимости размеров
        actual_gru_input_size = channels * height
        if actual_gru_input_size != self.gru_input_size:
            raise ValueError(f"Несоответствие размеров: ожидалось {self.gru_input_size}, получено {actual_gru_input_size}")
        
        # Переставляем размерности: (batch, time, channels, height)
        x = x.permute(0, 3, 1, 2)
        
        # Объединяем каналы и высоту: (batch, time, channels * height)
        x = x.contiguous().view(batch_size, width, channels * height)
        
        # RNN
        out, _ = self.gru(x)
        
        # Берем последний выход
        out = out[:, -1, :]
        
        # Fully connected
        out = self.fc(out)
        
        return out

# Функции обработки аудио
def load_audio(path, sr=SAMPLE_RATE, duration=DURATION):
    try:
        y, _ = librosa.load(path, sr=sr, mono=True, duration=duration)
        return y
    except Exception as e:
        print(f"Ошибка загрузки файла {path}: {str(e)}")
        return np.zeros(sr * duration)

def extract_enhanced_features(y, sr=SAMPLE_RATE, n_mfcc=N_MFCC, n_mels=N_MELS, hop_length=HOP_LENGTH):
    # MFCC
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length)
    
    # Mel spectrogram
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, hop_length=hop_length)
    mel_db = librosa.power_to_db(mel)
    
    # Chroma features
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=hop_length)
    
    # Spectral contrast
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=hop_length)
    
    # Tonnetz features
    tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
    
    # Stack features: shape (channels, time)
    feat = np.vstack([mfcc, mel_db, chroma, spectral_contrast, tonnetz])
    return feat.astype(np.float32)

def predict_audio(file_path, model, label2idx, idx2label, device=DEVICE):
    try:
        y = load_audio(file_path)
        features = extract_enhanced_features(y)
        max_time = 800
        if features.shape[1] < max_time:
            pad_width = max_time - features.shape[1]
            features = np.pad(features, ((0, 0), (0, pad_width)), mode='constant')
        else:
            features = features[:, :max_time]
        features = (features - features.mean()) / (features.std() + 1e-6)
        
        model.eval()
        with torch.no_grad():
            x = torch.tensor(features).unsqueeze(0).unsqueeze(0).to(device)
            output = model(x)
            probs = F.softmax(output, dim=1)
            pred_idx = output.argmax(dim=1).item()
            confidence = probs[0][pred_idx].item()
            
        return idx2label[pred_idx], confidence
    except Exception as e:
        raise RuntimeError(f"Ошибка обработки аудио: {str(e)}")